In [1]:
import copy

In [2]:
genesis = {
    "AMM":{"A1":100,"A2":100,"s":100,"fee":0.00},
    "Trader":{"A1":100,"A2":100,"s":0},
    "LP":{"A1":0,"A2":0,"s":100}
    }

In [3]:
def swapToAsset2(state, inputs):
    """
    Performs an asset swap in an Automated Market Maker (AMM) system. 
    The function simulates swapping Asset1 (A1) for Asset2 (A2) considering a given fee.

    Parameters:
    - state (dict): A dictionary representing the current state of the system.
        * state["AMM"]["A1"] represents the current quantity of Asset1 in the AMM.
        * state["AMM"]["A2"] represents the current quantity of Asset2 in the AMM.
        * state["AMM"]["fee"] represents the AMM's fee for the swap.
        * state[agent]["A1"] represents the current quantity of Asset1 with the agent.
        * state[agent]["A2"] represents the current quantity of Asset2 with the agent.
        
    - inputs (list): A list containing the following elements:
        * inputs[0]: A string representing the agent's name.
        * inputs[1]: A float representing the amount of Asset1 (dA1) the agent wishes to swap.
    
    Returns:
    - None: The function modifies the state dictionary in-place without returning any value.

    Note:
    The function first calculates the amount of Asset2 (dA2) the agent will receive in exchange for dA1.
    The amount is adjusted based on the AMM's fee. 
    The function then updates the quantities of Asset1 and Asset2 in both the AMM and the agent's state 
    if the swap conditions are met.
    """
    
    # Extract agent name and desired amount of Asset1 from inputs
    agent = inputs[0]
    dA1 = inputs[1]
    
    # Calculate the fee-adjusted amount of Asset2 (dA2) the agent will receive
    feeFactor = (1 - state["AMM"]["fee"])
    dA2 = state["AMM"]["A2"] / (state["AMM"]["A1"] + dA1 * feeFactor) * dA1 * feeFactor
    
    # Check if the swap conditions are met and update the state accordingly
    if dA1 > 0 and state[agent]["A1"] - dA1 >= 0:
        state["AMM"]["A1"] += dA1
        state[agent]["A1"] -= dA1
        state["AMM"]["A2"] -= dA2
        state[agent]["A2"] += dA2

In [4]:
def swapToAsset1(state, inputs):
    """
    Performs an asset swap in an Automated Market Maker (AMM) system. 
    The function simulates swapping Asset2 (A2) for Asset1 (A1) considering a given fee.

    Parameters:
    - state (dict): A dictionary representing the current state of the system.
        * state["AMM"]["A1"] represents the current quantity of Asset1 in the AMM.
        * state["AMM"]["A2"] represents the current quantity of Asset2 in the AMM.
        * state["AMM"]["fee"] represents the AMM's fee for the swap.
        * state[agent]["A1"] represents the current quantity of Asset1 with the agent.
        * state[agent]["A2"] represents the current quantity of Asset2 with the agent.
        
    - inputs (list): A list containing the following elements:
        * inputs[0]: A string representing the agent's name.
        * inputs[1]: A float representing the amount of Asset2 (dA2) the agent wishes to swap.
    
    Returns:
    - None: The function modifies the state dictionary in-place without returning any value.

    Note:
    The function first calculates the amount of Asset1 (dA1) the agent will receive in exchange for dA2.
    The amount is adjusted based on the AMM's fee. 
    The function then updates the quantities of Asset2 and Asset1 in both the AMM and the agent's state 
    if the swap conditions are met.
    """
    
    # Extract agent name and desired amount of Asset2 from inputs
    agent = inputs[0]
    dA2 = inputs[1]
    
    # Calculate the fee-adjusted amount of Asset1 (dA1) the agent will receive
    feeFactor = (1 - state["AMM"]["fee"])
    dA1 = state["AMM"]["A1"] / (state["AMM"]["A2"] + dA2 * feeFactor) * dA2 * feeFactor
    
    # Check if the swap conditions are met and update the state accordingly
    if dA2 > 0 and state[agent]["A2"] - dA2 >= 0:
        state["AMM"]["A2"] += dA2
        state[agent]["A2"] -= dA2
        state["AMM"]["A1"] -= dA1
        state[agent]["A1"] += dA1

In [5]:
!git add .
!git commit -a -m "evolve function"

/bin/sh: 1: git: not found
/bin/sh: 1: git: not found


In [6]:
def addLiquidity(state, inputs):
    """
    Adds liquidity to an Automated Market Maker (AMM) system.

    Parameters:
    - state (dict): A dictionary representing the current state of the system.
        * state["AMM"]["A1"] represents the current quantity of Asset1 in the AMM.
        * state["AMM"]["A2"] represents the current quantity of Asset2 in the AMM.
        * state["AMM"]["s"] represents the total supply of liquidity tokens in the AMM.
        * state[agent]["A1"] represents the current quantity of Asset1 with the agent.
        * state[agent]["A2"] represents the current quantity of Asset2 with the agent.
        * state[agent]["s"] represents the quantity of liquidity tokens held by the agent.
        
    - inputs (list): A list containing the following elements:
        * inputs[0]: A string representing the agent's name.
        * inputs[1]: A float representing the maximum amount of Asset1 (dA1) the agent wishes to add.
        * inputs[2]: A float representing the maximum amount of Asset2 (dA2) the agent wishes to add.
    
    Returns:
    - None: The function modifies the state dictionary in-place without returning any value.

    Note:
    The function calculates the amount of Asset1 (dA1) and Asset2 (dA2) the agent can add as liquidity.
    It checks if the agent has enough assets to add liquidity and if the input amounts are greater than zero.
    If the conditions are met, it updates the quantities of Asset1, Asset2, and liquidity tokens in both the AMM
    and the agent's state.
    """
    
    # Extract agent name, current AMM state, and input amounts
    agent = inputs[0]
    A1 = state["AMM"]["A1"]
    A2 = state["AMM"]["A2"]
    S = state["AMM"]["s"]
    max_dA1 = inputs[1]
    max_dA2 = inputs[2]
    
    # Calculate the actual amounts of Asset1 (dA1) and Asset2 (dA2) to add as liquidity
    dA1 = min(max_dA1, A1 / A2 * max_dA2)
    dA2 = min(max_dA2, A2 / A1 * max_dA1)
    
    # Check if the agent has enough assets to add liquidity and if the input amounts are greater than zero
    if (dA1 <= state[agent]["A1"] and dA2 <= state[agent]["A2"]) and (dA1 > 0 and dA2 > 0):
        # Update agent's asset and liquidity token balances
        state[agent]["A1"] -= dA1
        state[agent]["A2"] -= dA2
        state[agent]["s"] += min(dA1 / A1, dA2 / A2) * S
        
        # Update AMM's asset and liquidity token balances
        state["AMM"]["A1"] += dA1
        state["AMM"]["A2"] += dA2
        state["AMM"]["s"] += min(dA1 / A1, dA2 / A2) * S


In [7]:
def removeLiquidity(state, inputs):
    """
    Removes liquidity from an Automated Market Maker (AMM) and updates the agent's and AMM's state.

    Args:
        state (dict): The current state of the AMM and agents.
            Example state structure:
            {
                "AMM": {
                    "s": 1000,  # Total liquidity in the AMM
                    "A1": 500,   # Asset 1 balance in the AMM
                    "A2": 500    # Asset 2 balance in the AMM
                },
                "agent": {
                    "s": 100,   # Total liquidity owned by the agent
                    "A1": 50,   # Asset 1 balance owned by the agent
                    "A2": 50    # Asset 2 balance owned by the agent
                }
            }
        inputs (list): A list containing two elements.
            - inputs[0] (str): The agent's identifier.
            - inputs[1] (float): The amount of liquidity to remove from the AMM.

    Returns:
        None: This function modifies the 'state' dictionary in place and does not return a value.

    Note:
        - This function removes liquidity from the AMM and updates the agent's state.
        - It checks if the agent has sufficient liquidity and if the AMM has enough liquidity to remove.
        - The function then calculates the new AMM balances and updates the agent's balances accordingly.
        - It does not return a value but directly modifies the 'state' dictionary.
    """

    agent = inputs[0]
    dS = inputs[1]

    if dS > 0 and state[agent]["s"] - dS >= 0 and state["AMM"]["s"] - dS >= 0:
        # Calculate the proportional decrease in AMM balances
        DA = (1 - dS / state["AMM"]["s"])
        
        # Store initial AMM balances
        A1 = state["AMM"]["A1"]
        A2 = state["AMM"]["A2"]

        # Update agent's liquidity and asset balances
        state[agent]["s"] -= dS
        state[agent]["A1"] += A1 - state["AMM"]["A1"]
        state[agent]["A2"] += A2 - state["AMM"]["A2"]

        # Update AMM balances
        state["AMM"]["A1"] = A1 * DA
        state["AMM"]["A2"] = A2 * DA

        # Update total AMM liquidity
        state["AMM"]["s"] -= dS


In [8]:
genesis = {
    "AMM":{"A1":100,"A2":100,"s":100,"fee":0.0},
    "Trader":{"A1":100,"A2":100,"s":0},
    "LP":{"A1":0,"A2":0,"s":100}
    }

# from genesis , Trader agent will swap 10 of asset1 to get asset 2
swapToAsset2(genesis,["Trader",10])

genesis

{'AMM': {'A1': 110, 'A2': 90.9090909090909, 's': 100, 'fee': 0.0},
 'Trader': {'A1': 90, 'A2': 109.0909090909091, 's': 0},
 'LP': {'A1': 0, 'A2': 0, 's': 100}}

In [9]:
genesis = {
    "AMM":{"A1":100,"A2":100,"s":100,"fee":0.0},
    "Trader":{"A1":100,"A2":100,"s":0},
    "LP":{"A1":0,"A2":0,"s":100}
    }

# in a next state, Trader will exchange 10 Asset1 for Asset2
swapToAsset2(genesis,["Trader",10])

# in a next state, Trader will exchange 10 Asset2 for Asset1
swapToAsset1(genesis,["Trader",9.0909090909091])

genesis

{'AMM': {'A1': 99.99999999999999, 'A2': 100.0, 's': 100, 'fee': 0.0},
 'Trader': {'A1': 100.00000000000001, 'A2': 100.0, 's': 0},
 'LP': {'A1': 0, 'A2': 0, 's': 100}}

In [10]:
def nice_print(self):
    """
    A function for printing or formatting different types of data nicely.

    Args:
        self: Any data type (float, int, list, dict)

    Returns:
        Depending on the data type of 'self':
        - If 'self' is a float, it returns the rounded value with 3 decimal places.
        - If 'self' is an int, it returns the integer value unchanged.
        - If 'self' is a list, it prints each element of the list and returns None.
        - If 'self' is a dictionary, it prints each key-value pair and returns None.
    
    Note:
        - This function prints elements of a list or key-value pairs of a dictionary
          and returns None for those cases. If you want to capture the printed values,
          consider using a different approach like appending them to a list.
    """

    if type(self) == float:
        # If 'self' is a float, round it to 3 decimal places and return.
        return round(self, 3)

    if type(self) == int:
        # If 'self' is an int, return it unchanged.
        return self

    if type(self) == list:
        # If 'self' is a list, print each element and return None.
        list(map(lambda L: print(L), self))
        return None

    if type(self) == dict:
        # If 'self' is a dictionary, print each key-value pair and return None.
        dict(map(lambda kv: (kv[0], print(kv[1])), self.items()))
        return None

# Example usages:
# For floats and ints, it returns the value as specified.
# For lists and dictionaries, it prints the elements/pairs and returns None.

In [11]:
actionList = [
        [ removeLiquidity , [  "LP" , 5 ]],
        [ removeLiquidity , [  "LP" , 5 ]],
        [ removeLiquidity , [  "LP" , 5 ]]
]

In [12]:
actionList

[[<function __main__.removeLiquidity(state, inputs)>, ['LP', 5]],
 [<function __main__.removeLiquidity(state, inputs)>, ['LP', 5]],
 [<function __main__.removeLiquidity(state, inputs)>, ['LP', 5]]]

In [13]:
def evolve(state, actionStack):
    history = [copy.deepcopy(state)]
    for action in actionStack:
        action[0](state,action[1])
        history.append(copy.deepcopy(state))
    return history 

In [14]:
evolve(genesis, actionList)

[{'AMM': {'A1': 99.99999999999999, 'A2': 100.0, 's': 100, 'fee': 0.0},
  'Trader': {'A1': 100.00000000000001, 'A2': 100.0, 's': 0},
  'LP': {'A1': 0, 'A2': 0, 's': 100}},
 {'AMM': {'A1': 94.99999999999999, 'A2': 95.0, 's': 95, 'fee': 0.0},
  'Trader': {'A1': 100.00000000000001, 'A2': 100.0, 's': 0},
  'LP': {'A1': 0.0, 'A2': 0.0, 's': 95}},
 {'AMM': {'A1': 89.99999999999999, 'A2': 90.0, 's': 90, 'fee': 0.0},
  'Trader': {'A1': 100.00000000000001, 'A2': 100.0, 's': 0},
  'LP': {'A1': 0.0, 'A2': 0.0, 's': 90}},
 {'AMM': {'A1': 84.99999999999999, 'A2': 85.0, 's': 85, 'fee': 0.0},
  'Trader': {'A1': 100.00000000000001, 'A2': 100.0, 's': 0},
  'LP': {'A1': 0.0, 'A2': 0.0, 's': 85}}]

In [15]:
genesis = {
    "AMM":{"A1":100,"A2":100,"s":100,"fee":0.0},
    "Trader":{"A1":100,"A2":100,"s":0},
    "LP":{"A1":0,"A2":0,"s":100}
    }

actionList2 = [
        [ swapToAsset2 , [  "Trader" , 50 ]],
        [ swapToAsset1 , [  "Trader" , 25 ]],
]

!git commit -m "actionList2update"
evolve(genesis, actionList2)


/bin/sh: 1: git: not found


[{'AMM': {'A1': 100, 'A2': 100, 's': 100, 'fee': 0.0},
  'Trader': {'A1': 100, 'A2': 100, 's': 0},
  'LP': {'A1': 0, 'A2': 0, 's': 100}},
 {'AMM': {'A1': 150, 'A2': 66.66666666666667, 's': 100, 'fee': 0.0},
  'Trader': {'A1': 50, 'A2': 133.33333333333331, 's': 0},
  'LP': {'A1': 0, 'A2': 0, 's': 100}},
 {'AMM': {'A1': 109.0909090909091,
   'A2': 91.66666666666667,
   's': 100,
   'fee': 0.0},
  'Trader': {'A1': 90.9090909090909, 'A2': 108.33333333333331, 's': 0},
  'LP': {'A1': 0, 'A2': 0, 's': 100}}]

In [16]:
# the trader might be surprised because sum(A1,A2)1<sum(A1,A2)

In [17]:
# check_genesis_block that returns True if all inputs to the genesis block are valid, and False if there are invalid inputs. 
def check_genesis_block(genesis_block):
    """
    Check if all inputs to the custom Genesis block format are valid.

    Args:
        genesis_block (dict): The custom Genesis block format.
            Example structure:
            {
                "AMM": {
                    "A1": 100,
                    "A2": 100,
                    "s": 100,
                    "fee": 0.0
                },
                "Trader": {
                    "A1": 100,
                    "A2": 100,
                    "s": 0
                },
                "LP": {
                    "A1": 0,
                    "A2": 0,
                    "s": 100
                }
            }

    Returns:
        bool: True if all inputs are valid, False otherwise.

    Note:
        - This function checks the validity of the inputs in the custom Genesis block format.
        - It validates the structure and types of fields according to the provided format.
        - Additional checks can be added for specific requirements.
    """

def check_genesis_block(genesis_block):
    """
    Check if all inputs to the custom Genesis block format are valid.

    Args:
        genesis_block (dict): The custom Genesis block format.
            Example structure:
            {
                "AMM": {
                    "A1": 100,
                    "A2": 100,
                    "s": 100,
                    "fee": 0.0
                },
                "Trader": {
                    "A1": 100,
                    "A2": 100,
                    "s": 0
                },
                "LP": {
                    "A1": 0,
                    "A2": 0,
                    "s": 100
                }
            }

    Returns:
        bool: True if all inputs are valid and non-negative, False otherwise.

    Note:
        - This function checks the validity of the inputs in the custom Genesis block format.
        - It validates the structure, types, and non-negativity of fields according to the provided format.
        - Additional checks can be added for specific requirements.
    """

    # Check if the Genesis block contains the expected keys (AMM, Trader, LP)
    expected_keys = ["AMM", "Trader", "LP"]
    for key in expected_keys:
        if key not in genesis_block:
            return False

    # Check the structure, types, and non-negativity of subfields within each key
    for key, sub_data in genesis_block.items():
        if not isinstance(sub_data, dict):
            return False

        required_fields = ["A1", "A2", "s"]
        if key == "AMM":
            required_fields.append("fee")

        for field in required_fields:
            if field not in sub_data:
                return False

            if not isinstance(sub_data[field], (int, float)) or sub_data[field] < 0:
                return False

    # Additional checks can be added for specific requirements

    # If all checks pass, return True
    return True

# Example usage:
# genesis_block = {...}  # Provide your custom Genesis block data
# is_valid = check_genesis_block(genesis_block)
# print(is_valid)  # True if the Genesis block is valid and non-negative, False otherwise


In [18]:
genesis = {
    "AMM":{"A1":-100,"A2":100,"s":100,"fee":0.0},
    "Trader":{"A1":100,"A2":100,"s":0},
    "LP":{"A1":0,"A2":0,"s":100}
    }

check_genesis_block(genesis)

False

In [19]:
def update_pool_fees(state, updated_fee):
    # update the pool fees in the state
    state["AMM"]["fee"]=updated_fee
    
    return genesis

update_pool_fees(genesis, 1)

{'AMM': {'A1': -100, 'A2': 100, 's': 100, 'fee': 1},
 'Trader': {'A1': 100, 'A2': 100, 's': 0},
 'LP': {'A1': 0, 'A2': 0, 's': 100}}

In [20]:
def swapToAsset2_percent(state, inputs):
    """
    Swap a percent
    """
    # Extract agent name and desired amount of Asset1 from inputs
    agent = inputs[0]
    
    
    dA1 = state["AMM"]["Value"] * input[1]/100
    
#     dA1 = inputs[1]/state["AMM"]["A1"]
    
    # Calculate the fee-adjusted amount of Asset2 (dA2) the agent will receive
    feeFactor = (1 - state["AMM"]["fee"])
    dA2 = state["AMM"]["A2"] / (state["AMM"]["A1"] + dA1 * feeFactor) * dA1 * feeFactor
    
    # Check if the swap conditions are met and update the state accordingly
    if dA1 > 0 and state[agent]["A1"] - dA1 >= 0:
        state["AMM"]["A1"] += dA1
        state[agent]["A1"] -= dA1
        state["AMM"]["A2"] -= dA2
        state[agent]["A2"] += dA2

In [21]:
def swapToAsset1_percent(state, inputs):
    """
    Swap a percent
    """
    # Extract agent name and desired amount of Asset1 from inputs
    agent = inputs[0]
    
    # Calculate the amount of Asset2 (dA2) to swap as a percentage of the AMM's "Value"
    dA2 = state["AMM"]["Value"] * inputs[1] / 100
    
    # Calculate the fee-adjusted amount of Asset1 (dA1) the agent will receive in exchange for dA2
    feeFactor = (1 - state["AMM"]["fee"])
    dA1 = state["AMM"]["A1"] / (state["AMM"]["A1"] + dA2 * feeFactor) * dA2 * feeFactor
    
    # Check if the swap conditions are met and update the state accordingly
    if dA1 > 0 and state[agent]["A1"] - dA1 >= 0:
        state["AMM"]["A2"] += dA2
        state[agent]["A2"] -= dA2
        state["AMM"]["A1"] -= dA1
        state[agent]["A1"] += dA1

In [22]:
def invariant(state):
    return state["AMM"]["A1"] * state["AMM"]["A2"]

def asset1(state):
    return state["AMM"]["A1"]+state["Trader"]["A1"]+state["LP"]["A1"]

def asset2(state):
    return state["AMM"]["A2"]+state["Trader"]["A2"]+state["LP"]["A2"]

In [23]:
genesis = {
    "AMM":{"A1":100,"A2":100,"s":100,"fee":0.0},
    "Trader":{"A1":100,"A2":100,"s":0},
    "Liquidator":{"A1":0,"A2":0,"s":100}
    }

state = copy.deepcopy(genesis)

swapToAsset2(state,["Trader",13])

invariant(genesis)==invariant(state)

True

In [24]:
# change liquidator by liquidity provider
genesis = {
    "AMM":{"A1":100,"A2":100,"s":100,"fee":0.0},
    "Trader":{"A1":100,"A2":100,"s":0},
    "LP":{"A1":0,"A2":0,"s":100}
    }

state = copy.deepcopy(genesis)

swapToAsset1(state,["Trader",13])

asset1(genesis)==asset1(state) and asset2(genesis)==asset2(state)

True

In [25]:
genesis = {
    "AMM":{"A1":100,"A2":100,"s":100,"fee":0.0},
    "Trader":{"A1":100,"A2":100,"s":0},
    "LP":{"A1":0,"A2":0,"s":100}
    }

state = {
    "AMM":{"A1":100,"A2":100,"s":100,"fee":0.0},
    "Trader":{"A1":0,"A2":100,"s":0},
    "LP":{"A1":0,"A2":0,"s":100}
    }
    
asset1(genesis)==asset1(state) and asset2(genesis)==asset2(state)

False

In [26]:
genesis = {
    "AMM":{"A1":100,"A2":100,"s":100,"fee":0.0},
    "Trader":{"A1":100,"A2":100,"s":0},
    "Liquidator":{"A1":0,"A2":0,"s":100}
    }

state = copy.deepcopy(genesis)

swapToAsset1(state,["Trader",13])

gained = state["Trader"]["A1"] - genesis["Trader"]["A1"]

swapToAsset2(state,["Trader",gained])

genesis == state

True

ModuleNotFoundError: No module named 'matplotlib'

In [30]:
import math
import matplotlib.pyplot as plt
genesis = {
    "AMM":{"A1":99,"A2":1,"s":math.sqrt(99),"fee":0.0},
    "Trader":{"A1":1,"A2":99,"s":0},
    "LP":{"A1":200,"A2":200,"s":200}
    }

actionList= [ [swapToAsset1,["Trader",1]] ] * 99

history = evolve(genesis, actionList)

AMM_r1=[]
Trader_r1=[]
AMM_r2=[]
Trader_r2=[]
for s in history:
    AMM_r1.append(s["AMM"]["A1"])
    Trader_r1.append(s["Trader"]["A1"])
    AMM_r2.append(s["AMM"]["A2"])
    Trader_r2.append(s["Trader"]["A2"])   


plt.figure(figsize=(10,4)) 
    
plt.subplot(1, 2, 1) 
plt.plot(AMM_r1,AMM_r2)
plt.title('AMM')
plt.xlabel("Asset 1")
plt.ylabel("Asset 2")

plt.subplot(1, 2, 2)
plt.plot(Trader_r1,Trader_r2)
plt.title('Trader')
plt.xlabel("Asset 1")
plt.ylabel("Asset 2")

ModuleNotFoundError: No module named 'matplotlib'

In [35]:
!pip install matplotlib

  Using cached https://files.pythonhosted.org/packages/ad/62/7b662284352867a86acfb636761ba351723fc3a235efd8397578d903413d/matplotlib-3.5.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/e3/d9/e9bae85e84737e76ebbcbea13607236da0c0699baed0ae4f1151b728a608/fonttools-4.38.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/00/d5/4903f310765e0ff2b8e91ffe55031ac6af77d982f0156061e20a4d1a8b2d/Pillow-9.5.0.tar.gz
  ERROR: Complete output from command /snap/jupyter/6/bin/python -u -c 'import setuptools, tokenize;__file__='"'"'/tmp/pip-install-5pl_awzf/pillow/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-1m8robbl --python-tag cp37:
  ERROR: running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.linux-x86_64-3.7
  creating

Failed to build pillow
  Running setup.py install for pillow ... error
    ERROR: Complete output from command /snap/jupyter/6/bin/python -u -c 'import setuptools, tokenize;__file__='"'"'/tmp/pip-install-5pl_awzf/pillow/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-ftl4hj07/install-record.txt --single-version-externally-managed --compile --user --prefix=:
    ERROR: running install
    running build
    running build_py
    creating build
    creating build/lib.linux-x86_64-3.7
    creating build/lib.linux-x86_64-3.7/PIL
    copying src/PIL/IcnsImagePlugin.py -> build/lib.linux-x86_64-3.7/PIL
    copying src/PIL/ImageOps.py -> build/lib.linux-x86_64-3.7/PIL
    copying src/PIL/JpegPresets.py -> build/lib.linux-x86_64-3.7/PIL
    copying src/PIL/ImImagePlugin.py -> build/lib.linux-x86_64-3.7/PIL
    copying src/PIL/EpsImagePlug

ERROR: Command "/snap/jupyter/6/bin/python -u -c 'import setuptools, tokenize;__file__='"'"'/tmp/pip-install-5pl_awzf/pillow/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-ftl4hj07/install-record.txt --single-version-externally-managed --compile --user --prefix=" failed with error code 1 in /tmp/pip-install-5pl_awzf/pillow/


In [39]:
from matplotlib import pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [41]:
import math

In [44]:
!pip list

Package            Version
------------------ -------
charset-normalizer 3.1.0  
cycler             0.11.0 
fonttools          4.38.0 
greenlet           2.0.2  
idna               3.4    
importlib-metadata 6.6.0  
kiwisolver         1.4.5  
langchain          0.0.27 
numpy              1.21.6 
packaging          23.2   
pydantic           1.10.7 
pyparsing          3.1.1  
python-dotenv      0.21.1 
PyYAML             6.0    
requests           2.29.0 
SQLAlchemy         2.0.12 
typing-extensions  4.5.0  
urllib3            1.26.15
zipp               3.15.0 
